In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import json
import collections
from lxml import etree
from xml.etree.ElementTree import XML
from random import shuffle
import pathlib
import itertools
import numpy as np
import utils

docxFileName = "../resources/quicks/quick_section4.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

### Using xpath to find main stations

In [ ]:
mainstation = ""
lowerstation = ""
dText = dict()
counter = 0
for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text])
    mainstation, counter = utils.is_mainst(para, mainstation, counter, ns)
    print(mainstation)
    description = description.lstrip('\x01').strip()
    if description:
        if (counter, mainstation) in dText:
            dText[(counter, mainstation)].append(description)
        else:
            description = re.sub('^(' + re.escape(mainstation) + ')', '\1', description).lstrip('\x01').strip()
            description = re.sub(r" +", " ", description).lstrip('\x01').strip()
            if description:
                dText[(counter, mainstation)] = [description]

### Indexing main stations

In [ ]:
dStations = collections.OrderedDict(dText)

indices = []
stations = []
descriptions = []
for k in dStations:
    indices.append(k[0])
    stations.append(k[1])
    descriptions.append(dStations[k])

stationdf = pd.DataFrame(columns=["Index", "Station", "Description"])
stationdf["Index"] = indices
stationdf["Station"] = stations
stationdf["Description"] = descriptions
stationdf = stationdf.set_index("Index")

### Using regex to find substations

In [ ]:
stations = pd.DataFrame(columns=['station','type','description'])
cols = ['MainId', 'MainStation', 'SubId', 'SubStation', 'Description']
lst = []
subInd = 0
for i, row in stationdf.iterrows():
    main_station = row["Station"]
    description = row["Description"]
    dSubstations, subInd = utils.process_decription(main_station, description, subInd)
    for ss in dSubstations:
        lst.append([i, main_station, ss[0], ss[1], dSubstations[ss]])
subsdf = pd.DataFrame(lst, columns=cols)

### Renaming abbreviated substations

In [ ]:
subsdf['SubStFormatted'] = subsdf.apply(lambda row: utils.subst_rename(row["MainStation"], row["SubStation"]), axis = 1)
subsdf = subsdf[["MainId", "SubId", "MainStation", "SubStation", "SubStFormatted", "Description"]]
subsdf.to_pickle('quicks_processed.pkl')
subsdf.to_csv('quicks_processed.tsv', sep="\t", index=False)

### Formatting station names as required by DeezyMatch

In [ ]:
unique_placenames_array = list(set(list(np.array(subsdf["MainStation"]))))
utils.format_for_candranker("../toponym_matching/toponyms/quicks_mainst_queries", unique_placenames_array)

unique_placenames_array = list(set(list(np.array(subsdf["SubStFormatted"]))))
utils.format_for_candranker("../toponym_matching/toponyms/quicks_subst_queries", unique_placenames_array)